In [188]:
import json
import requests
import collections
from datetime import date, timedelta
import statistics

In [189]:
with open("./config_secret.json") as key_file:
    key = json.load(key_file)

In [190]:
MYAPIKEY = key['api_key']

In [191]:
TICKERCODE = 'AFX_X';
START_DATE = (datetime.date(2017, 1, 1)).strftime('%Y-%m-%d');
END_DATE = (datetime.date(2017, 12, 31)).strftime('%Y-%m-%d');
url = 'https://www.quandl.com/api/v3/datasets/FSE/'+TICKERCODE
params = {'start_date': START_DATE , 'end_date': END_DATE , 'api_key': MYAPIKEY }

response = requests.get(url = url, params = params)

In [194]:
if(response.status_code == 200):
    # get data for yeae 2017
    data = json.loads(response.content.decode('utf8'))
    for item in data.values():
        largest_change = -1
        inter_day_largest_change = -1
        prev_day_close_price = -1
        total_traded_volume = 0
        count_traded_volume = 0;
        
        open_price_col_index = item['column_names'].index('Open')
        high_price_col_index = item['column_names'].index('High')
        low_price_col_index = item['column_names'].index('Low')
        close_price_col_index = item['column_names'].index('Close')
        traded_vol_col_index = item['column_names'].index('Traded Volume')
        date_vol_col_index = item['column_names'].index('Date')
        
        tradingVolumeList = []
        openPriceList = []
        
        
        price_data_list = item['data'];
        
        for price_data in price_data_list:
            if(type(price_data[traded_vol_col_index]) is not float):
                local_traded_vol_data = 0.0
            else:
                local_traded_vol_data = price_data[traded_vol_col_index]
                tradingVolumeList.append(local_traded_vol_data)
                
            total_traded_volume = total_traded_volume + local_traded_vol_data
            count_traded_volume = count_traded_volume +1
            
            if(type(price_data[close_price_col_index]) is not float):
                local_close_price_data = 0.0
            else:
                local_close_price_data = price_data[close_price_col_index]
                
            if(inter_day_largest_change == -1):
                inter_day_largest_change = local_close_price_data
            elif((local_close_price_data - prev_day_close_price) > inter_day_largest_change):
                inter_day_largest_change = local_close_price_data - prev_day_close_price
            prev_day_close_price = local_close_price_data
                
                
            #find largest change in any one day for year 2017
            
            if(type(price_data[high_price_col_index]) is not float):
                local_high_price_data = 0.0
            else:
                local_high_price_data = price_data[high_price_col_index]
            
            if(type(price_data[low_price_col_index]) is not float):
                local_low_price_data = 0.0
            else:
                local_low_price_data = price_data[low_price_col_index]
                
            if (largest_change == -1):
                largest_change = local_high_price_data - local_low_price_data
            else:
                if((local_high_price_data - local_low_price_data) > largest_change):
                    largest_change = local_high_price_data - local_low_price_data
                
            #find highest in a year of 2017 and lowest in a year of 2017  
            if(type(price_data[open_price_col_index]) is float):
                openPriceList.append(price_data[open_price_col_index])
                
                
        print("highest opening price : {}".format(max(openPriceList)))
        print("lowest opening price : {}".format(min(openPriceList)))
        print("largest change in any one day : {}".format(largest_change))
        print("largest inter-day change : {}".format(inter_day_largest_change))
        print("Average daily trading volume : {}".format(total_traded_volume / count_traded_volume))
        print("Median traded volumne : {} ".format(statistics.median(tradingVolumeList)))
                
        #for key in item.keys():
        #    print(key)
else:
    print(
        "Error in getting data from website : {} ; status code : {}"
        .format(
            response.request.url,
            response.status_code
        )
    )

highest opening price : 53.11
lowest opening price : 34.0
largest change in any one day : 2.8100000000000023
largest inter-day change : 51.76
Average daily trading volume : 89124.33725490196
Median traded volumne : 76286.0 
